<a href="https://colab.research.google.com/github/olgasem10/Unsafe/blob/main/Optuna_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers.trainer_callback import EarlyStoppingCallback
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np
import pandas as pd
import optuna
from typing import Dict

### Данные (с добавлением спец токена)

In [ ]:
train_df = pd.read_csv('/kaggle/input/unsafe/train_randst0.csv')
test_df = pd.read_csv('/kaggle/input/unsafe/val_randst0.csv')
#train_df = train_df[["text", "unsafe"]]
#test_df = test_df[["text", "unsafe"]]

In [ ]:
train_df = train_df.loc[(train_df['unsafe'] >= 0.8) | (train_df['unsafe'] <= 0.2)]

In [ ]:
translator = {
    'crime_real' : 'криминал',
    'crime_web' : 'веб-криминал',
    'drugs' : 'наркотики',
    'gambling' : 'азартные игры',
    'pornography' : 'порнография',
    'prostitution' : 'проституция',
    'slavery' : 'рабство',
    'suicide' : 'суицид',
    'terrorism' : 'терроризм',
    'weapons' : 'оружие', 
    'body_shaming' : 'уродства',
    'halth_shaming' : 'инвалидность',
    'politics' : 'политика',
    'racism' : 'расизм',
    'religion' : 'религия',
    'sex_minorities' : 'гомосексуализм',
    'sexism' : 'сексизм',
    'social' : 'социальное',
    'none' : ''
}

In [ ]:
col_names = ['crime_real',
 'crime_web',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'halth_shaming',
 'politics',
 'racism',
 'religion',
 'sex_minorities',
 'sexism',
 'social']

In [ ]:
def binary(val):
    return round(val)

In [ ]:
for name in col_names:
    train_df[name] = train_df[name].apply(binary)
    test_df[name] = test_df[name].apply(binary)

In [ ]:
train_df['unsafe'] = train_df['unsafe'].apply(binary)
test_df['unsafe'] = test_df['unsafe'].apply(binary)

In [ ]:
def add_special_tokens(dataframe):
    new_dataframe = dataframe.copy()
    for column in col_names:
        indices = dataframe.index[dataframe[column] == 1]
        new_dataframe.loc[indices, ['text']] = '<' + translator[column] + '> ' + dataframe.loc[indices, ['text']].astype(str)
    return new_dataframe

In [ ]:
train_df = add_special_tokens(train_df)
test_df = add_special_tokens(test_df)

In [ ]:
#train_df = train_df.sample(frac=1)
#test_df = test_df.sample(frac=1)

In [ ]:
x_train = train_df['text'].tolist()#[:15000]
y_train = train_df['unsafe'].tolist()#[:15000]
x_test = test_df['text'].tolist()#[:2000]
y_test = test_df['unsafe'].tolist()#[:2000]

In [ ]:
len(x_test)

In [ ]:
class UnsafeData(Dataset):

    def __init__(self, texts, targets, tokenizer, max_len):
        
        super().__init__()
        
        self.texts = texts
        self.targets = targets        
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        
        return len(self.texts)

    
    def __getitem__(self, index):
        x = self.texts[index]

        enc_dict = self.tokenizer(x, truncation=True, max_length=self.max_len, padding='max_length')
      
        item = {key: torch.tensor(val).long() for key, val in enc_dict.items()}
        item['labels'] = torch.tensor(self.targets[index]).long()

        return item 

### Обучение

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased-conversational'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name)

In [ ]:
train_dataset = UnsafeData(x_train, y_train, tokenizer, max_len = 60)
test_dataset = UnsafeData(x_test, y_test, tokenizer, max_len = 60)

In [ ]:
len(train_dataset), len(test_dataset)

(120120, 24501)

In [ ]:
train_dataset[10000]

{'input_ids': tensor([  101,   160, 21749,   130, 48931,   164, 40051,   477,   507,   465,
         58496,   439, 28019, 14646, 32260,  8297, 41387,   128,  1007,  1151,
         19644,   340,   362,  5703,   166,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor(1)}

In [ ]:
def model_init():
    return BertForSequenceClassification.from_pretrained(model_name, return_dict=True)

In [ ]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/bert1',
    logging_steps = 600,
    evaluation_strategy = 'steps',
    logging_dir='/kaggle/working/bert1/logs',
    save_steps = 10000,
    save_total_limit = 2,
    #metric_for_best_model  = 'f1',
    #greater_is_better = True,
    #load_best_model_at_end = True
)

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
### По какой метрике оптимизировать

def my_func(metrics):
    return metrics['eval_f1']

### TPESampler

In [ ]:
### Функция для пространства парметров

def my_hp_space_optuna(trial) -> Dict[str, float]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 1), #1, 2
        #"seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1)
    }

In [ ]:
best_run = trainer.hyperparameter_search(
    hp_space = my_hp_space_optuna,
    n_trials=5,
    #n_jobs = 2,
    compute_objective = my_func,
    direction="maximize",
    backend = 'optuna',
    study_name = 'Example',
    sampler = optuna.samplers.TPESampler(),
    pruner=optuna.pruners.SuccessiveHalvingPruner() #??? #https://optuna.readthedocs.io/en/stable/reference/pruners.html#module-optuna.pruners
)

[I 2021-03-18 12:50:56,005] A new study created in memory with name: Example
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.412600,0.441988,0.794376,0.791398,0.789764,0.794376,56.824800,431.168000
1200,0.364900,0.436693,0.803151,0.795892,0.796062,0.803151,56.960500,430.140000
1800,0.346800,0.422597,0.808375,0.805907,0.804549,0.808375,56.947100,430.241000


[I 2021-03-18 13:07:32,900] Trial 0 finished with value: 0.8059069203023017 and parameters: {'learning_rate': 1.4117764203025751e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 64, 'weight_decay': 0.07512942141733155}. Best is trial 0 with value: 0.8059069203023017.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.584200,0.537411,0.719236,0.635839,0.718963,0.719236,56.784900,431.470000
1200,0.477100,0.492998,0.758622,0.755585,0.753504,0.758622,56.648600,432.509000
1800,0.445900,0.481442,0.767152,0.764239,0.762289,0.767152,57.067000,429.338000
2400,0.422400,0.478255,0.772866,0.767923,0.765690,0.772866,56.247600,435.592000
3000,0.424500,0.472778,0.774826,0.771304,0.769269,0.774826,56.520200,433.491000
3600,0.413700,0.466059,0.779274,0.773598,0.771638,0.779274,56.815900,431.235000
4200,0.407900,0.467155,0.780703,0.773159,0.771856,0.780703,57.134000,428.834000
4800,0.407900,0.465306,0.782458,0.774034,0.773312,0.782458,57.088300,429.177000
5400,0.394900,0.465390,0.779478,0.780114,0.780812,0.779478,57.065400,429.349000
6000,0.399900,0.460889,0.782539,0.780002,0.778338,0.782539,57.320300,427.440000


[I 2021-03-18 13:39:14,688] Trial 1 finished with value: 0.7802529324828464 and parameters: {'learning_rate': 1.2779229703135447e-06, 'num_train_epochs': 1, 'per_device_train_batch_size': 16, 'weight_decay': 0.03047893234751622}. Best is trial 0 with value: 0.8059069203023017.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2021-03-18 14:00:39,261] Trial 2 finished with value: 0.8050659830965247 and parameters: {'learning_rate': 4.7608224975319416e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 32, 'weight_decay': 0.04178583915566045}. Best is trial 0 with value: 0.8059069203023017.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.464300,0.470469,0.775070,0.776287,0.777729,0.775070,56.944400,430.261000
1200,0.418500,0.454358,0.777887,0.781758,0.788487,0.777887,56.891400,430.663000
1800,0.396600,0.442392,0.789192,0.776145,0.780528,0.789192,56.554800,433.226000
2400,0.377700,0.436637,0.796825,0.791749,0.790343,0.796825,57.231900,428.100000
3000,0.376600,0.464693,0.795723,0.785387,0.787657,0.795723,57.215100,428.226000
3600,0.376200,0.476868,0.796294,0.782457,0.789386,0.796294,56.812700,431.259000
4200,0.361200,0.442924,0.800620,0.798823,0.797582,0.800620,56.943800,430.266000
4800,0.365800,0.449968,0.800294,0.794718,0.793655,0.800294,56.946600,430.245000
5400,0.351200,0.447620,0.803967,0.803732,0.803508,0.803967,57.191800,428.400000
6000,0.356500,0.420388,0.807069,0.804234,0.802817,0.807069,56.942500,430.276000


[I 2021-03-18 14:32:20,009] Trial 3 finished with value: 0.8069978438396579 and parameters: {'learning_rate': 2.9489250021210324e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 16, 'weight_decay': 0.016346936245910295}. Best is trial 3 with value: 0.8069978438396579.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.480400,0.470405,0.772213,0.769153,0.767207,0.772213,56.738600,431.823000
1200,0.401500,0.464092,0.785274,0.778192,0.776969,0.785274,57.083700,429.212000
1800,0.391600,0.441448,0.791192,0.785412,0.783987,0.791192,56.643600,432.547000
2400,0.381900,0.442578,0.796008,0.790612,0.789275,0.796008,56.819700,431.206000
3000,0.367400,0.440015,0.792866,0.793205,0.793564,0.792866,56.520200,433.491000
3600,0.365800,0.435888,0.797029,0.795190,0.793909,0.797029,56.947200,430.240000


[I 2021-03-18 14:53:41,614] Trial 4 finished with value: 0.7951903321277392 and parameters: {'learning_rate': 4.240225969679652e-06, 'num_train_epochs': 1, 'per_device_train_batch_size': 32, 'weight_decay': 0.020472146538012794}. Best is trial 3 with value: 0.8069978438396579.


Результат: Best is trial 3 with value: 0.8069978438396579

parameters: {'learning_rate': 2.9489250021210324e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 16, 'weight_decay': 0.016346936245910295}

#### GridSampler

In [ ]:
search_space = {
        "learning_rate": [1e-6, 1e-4],
        "num_train_epochs": [1, 2],
        "seed": [32, 40],
        "per_device_train_batch_size": [4, 8, 16, 32, 64],
        'weight_decay': [0, 0.01, 0.001]
    }

In [ ]:
best_run = trainer.hyperparameter_search(
    n_trials=2,
    compute_objective = my_func,
    direction="maximize",
    backend = 'optuna',
    study_name = 'Example',
    sampler = optuna.samplers.GridSampler(search_space),
    pruner=optuna.pruners.SuccessiveHalvingPruner() #??? #https://optuna.readthedocs.io/en/stable/reference/pruners.html#module-optuna.pruners
)

[I 2021-03-18 15:00:10,681] A new study created in memory with name: Example
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.476000,0.580707,0.766010,0.763432,0.761612,0.766010,57.027600,429.634000
1200,0.559400,0.544872,0.732378,0.734262,0.736504,0.732378,56.898200,430.611000
1800,0.536300,0.557038,0.740704,0.730686,0.727560,0.740704,56.847000,430.999000
2400,0.517600,0.533692,0.745888,0.739857,0.736798,0.745888,57.007300,429.787000
3000,0.497500,0.548778,0.754500,0.749721,0.747064,0.754500,57.126400,428.891000
3600,0.481500,0.498183,0.762785,0.761687,0.760730,0.762785,57.056700,429.415000


[I 2021-03-18 15:21:35,597] Trial 0 finished with value: 0.7616865427144448 and parameters: {'learning_rate': 0.0001, 'num_train_epochs': 1, 'seed': 40, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.7616865427144448.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-conversational and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
600,0.558000,0.516291,0.746133,0.725684,0.728884,0.746133,56.784200,431.475000
1200,0.462200,0.485045,0.759928,0.760197,0.760476,0.759928,56.627400,432.671000
1800,0.432700,0.472674,0.770785,0.767325,0.765245,0.770785,57.381200,426.987000
2400,0.414200,0.468135,0.774336,0.770978,0.768973,0.774336,56.675200,432.306000
3000,0.408400,0.464955,0.774907,0.772908,0.771422,0.774907,56.590700,432.951000
3600,0.404100,0.464337,0.775642,0.773421,0.771832,0.775642,56.165000,436.233000


[I 2021-03-18 15:54:41,419] Trial 1 finished with value: 0.7734213576011032 and parameters: {'learning_rate': 1e-06, 'num_train_epochs': 2, 'seed': 40, 'per_device_train_batch_size': 64}. Best is trial 1 with value: 0.7734213576011032.


In [ ]:
#!rm -r ./bert1

In [ ]:
trainer.save_model('rubert-base-cased-conversational-1')